<a href="https://colab.research.google.com/github/arumajirou/-daily-test/blob/main/Anomaly_Transformer_in_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Anomaly Transformer:アソシエーション(連合,連想)の不一致を利用した時系列異常の検出**


---
- 時系列における**異常点**の**教師なし検出**は，モデルに**識別可能な基準**を導き出すことが要求される難しい問題である． 

- 従来の手法は，主にポイントワイズ(点別の)表現やペアワイズアソシエーション(二つ一組の比較)の学習によってこの問題に取り組んできたが，どちらも複雑なダイナミクス(挙動)を推論するには不十分であった．

- 近年、Transformerはポイントワイズ(点ごとの)表現とペアワイズアソシエーション(二つ一組の比較)の**統一的なモデリング(計算方法)**に大きな力を発揮しており、我々は**各時点の自己注視度分布**が**系列全体**との**豊かな関連性**を具現化できることを見出した。

- 我々の重要な観測は、**異常値の希少性**により、異常値から系列全体への非自明な関連付けを構築することが極めて困難であり、その結果、異常値の関連付けは主にその**隣接する時点に集中**することである。

- この**隣接集中の偏り**は、**正常点**と**異常点**を本質的に**区別できる**ような関連付けに基づく基準を意味し、我々はこれを**「関連性不一致」**という言葉で強調する。

- 技術的には、**連想不一致**を計算するための新しいメカニズムとして、㈶異常変換器 "を提案します。

- 関連性不一致の正規・非正規区別性を増幅させるために、**ミニマックス戦略**を考案しています。

- Anomaly Transformerは、サービス監視、宇宙・地球探査、水処理という3つのアプリケーションの教師なし時系列異常検出ベンチマーク6件において、最先端の結果を達成した。

---
- **論文**　Anomaly Transformer:アソシエーションの不一致を利用した時系列異常の検出
https://arxiv.org/abs/2110.02642

- ICLR2022の改訂を反映したモデル
https://github.com/spencerbraun/anomaly_transformer_pytorch/tree/main/archive

# **時系列異状検知にもTransformer**
### **3つの要点**

- いよいよ多変量時系列異状検知にもTransformerが現れました

- グラフも含めた深層学習により多変量の時系列の表現力は向上してきましたが、まだ単一時点に限ります

- Transformerのグローバルおよび長期の連関に対しての表現力を活かして、改造したAnomaly-Attentionを含む2分岐の構造で従来のSOTAを超える性能を確認しています

---
## **はじめに**
- いよいよ**多変量時系列異状検知**にも**Transformer**が現れました。

- 実際には、この論文の前に**GTA (Chen et al., 2021)vというグラフ構造により複数のIoTセンサーの関係を学習するモデル**にTransformerが時間軸のモデリングと異状検知の再構築基準に用いられていました。

- ブロック図を添付しておきます。他にも**TranAD**、**TiSAT**など続々と発表されています。

- 機会があれば、これらについても紹介したいと思います。

<img src="https://aisholar.s3.ap-northeast-1.amazonaws.com/post_images/4668dfff88e305d6790f787f3641c6da.png">

- 今回紹介するAnomaly Transformerは、**セルフアテンションメカニズムを異状検知用に改造したもの**です。

- 以前の記事の繰り返しになりますが、実世界のデータでは、異状検知を行いたい場合に、異常のデータはまれであり、**ラベリングを行うことは困難**であるため、ここでは**教師なし学習にフォーカス**します。

- この場合、教師データなしに、**正常／異常の判別基準**を作る必要があります。古典的な手法では、
 - **密度見積もり**
 - **クラスタリング手法**

がありますが、

- これらは**時間的要素を考慮しておらず**、見えていない実際のシナリオ(場面変化の順序)に一般化するのは難しいです。

- ニューラルネットワークによる表現学習の能力を利用して、最近の深層学習モデルは優れた成果を出しています。

- 主な手法カテゴリーでは、**回帰型ネットワーク**を通して**点的に表現学習**し、**再構成**あるいは**自己回帰タスク**により**自己教師学習**します。

- やはり、**異常データの少なさ**の為、**複雑な時間軸パターン**に対しては、**判別が難しく**なっています。

- また、**再構成**あるいは**予測誤差**は**時点毎**に計算されるので、**時間的なコンテキスト(前後の脈絡)の包括的な表現は難しく**なっています。

---
- 別の手法カテゴリーでは、**明示的な連関(association)モデリング**により異状を検知します。

 - **ベクトル自己回帰(VAR)** VARは、時系列データに対して、**変数間の相関**や**影響**を**推定する**ことができます。
 - **状態空間モデル**は**系の振る舞いを表す数学的モデルの一種**
 - **グラフニューラル ネットワーク(GNN)**グラフとして表現できるデータを処理するための人工ニューラル ネットワーク
 がこのカテゴリーに属します。

- 以前紹介したように、グラフとして表現できるデータを処理するための人工ニューラル ネットワークは多変量時系列の動的なグラフの学習に適用されています。

- 表現能力は向上していますが、まだ**単一時点のモデル**に限定されています。

- 一方**部分シーケンスベースの手法**は、**部分シーケンス間の類似性**を計算することにより**異常＊＊*を検知します。

- しかし、これらの手法(部分シーケンスベースの手法)では、**各時点と系列全体の詳細な時間的連関を捕捉すること**ができません。

---

- この論文の手法では、Transformerを時系列異状検知の教師なし学習に適用しています。

- Transformerがひろく適用されているのは、**グローバルな表現**と、**長期の連関を統一的に表現できる**ためです。

- 時系列に適用した場合に、セルフアテンションマップは**各時点の時間的連関を表現**します。

- これを**系列連関(series-association)**と名付けます。

- さらに、**異常の希少性**と**正常なパターンの優位性**のために、異常が**系列全体との強い関連を構築すること**はより困難であることが観察されます。

- 異常の関連は、**連続性**のために**同様の異常パターン**を含む可能性が高い隣接する時点に集中する必要があります。

- このような隣接濃度の誘導バイアスは、**事前連関(prior-association)**と呼ばれます。

- 対照的に、支配的な正常の時点は、**隣接する領域**に限定されるのではなく、**系列全体との有益な関連**を発見することができます。

- この観察に基づいて、**関連分布の固有の正常-異常識別可能性**を利用しようとします。

- これにより、**各時点の新しい異常基準**が導き出されます。

- これは、**各時点の事前連関**とその**系列連関の間の距離**によって**定量化**され、**連関不一致(Association Discrepancy)**と呼ばれます。

- 前述のように、**異常の関連は隣接集中**している可能性が高いため、**異常は正常の時点よりも連関不一致が小さくなります。**

---
- 教師なし時系列異常検出にTransformerを導入し、連関学習用のAnomalyTransformerを提案しています。

- **連関不一致(Association Discrepancy)**を計算するために、セルフアテンションメカニズムをAnomaly-Attentionに刷新します。

- これには、**各時点の事前連関**と**系列連関**を**それぞれモデル化**する**2つの分岐構造**が含まれています。

- **事前連関**では、学習可能なガウスカーネルを使用して、**各時点の隣接する濃度の誘導バイアスを提示**します。

- 一方、**系列連関**は、**生の系列**から学習した**セルフアテンションの重み**に対応します。

- さらに、**ミニマックス戦略**が**2つの分岐間**に適用されます。

- これにより、**連関不一致**の**正常と異常の識別可能性が増幅**され、**新しい連関ベースの基準**がさらに導き出されます。 


---

この論文の貢献は、次の3つです。

- **連関の不一致**の主要な観察に基づいて、Anomaly-Attentionメカニズムを備えたAnomaly-Transformerを提案します。

 - これにより、**事前連関**と**系列連関**を**同時にモデル化**して、**連関の不一致を具体化**できます。

- **連関不一致**の**正常**と**異常**の**識別可能性を拡大**し、さらに**新しい連関ベースの検出基準を導出**するための**ミニマックス戦略**を提案します。

- **Anomaly Transformer**は、3つの実際のアプリケーションの**6つのベンチマークでSOTAの異状検出結果**を実現します。 
 - 広範な切り分けと洞察に満ちたケーススタディを行います。

---
# **まとめ**
この論文は、**教師なし時系列異常検出問題**を研究しています。 

- 以前の方法とは異なり、Transformersによる**より有益な時間軸の連関を学習**します。 

- 重要である**連関不一致の観察**に基づいて、Anomaly Transformerを提案しています。

- これには、**連関不一致を具体化**するための**2分岐構造**の**Anomaly-Attention**が含まれます。 

- **ミニマックス戦略**は、**正常な時点と異常な時点の違い**を**さらに増幅する**ために採用されています。

**連関不一致**を導入することにより、**再構成の性能**と**連関不一致を連携させる連関ベースの基準**を提案します。 

- Anomaly Transformerは、経験的研究のデータセットで広く評価し、**SOTAの結果を確認**しています。

- 将来的には、**自己回帰**や**状態空間モデル**の古典的な分析に照らして、Anomaly Transformerの理論的な研究を行うとしています。

---

- 時系列異状検知にもTransformer

https://ai-scholar.tech/articles/error-detection/Anomaly_Transforme

# **ベクトル自己回帰（VectorAutoregression, VAR）**

---
- 時系列データを分析するための方法の一つです。VARは、時系列データに対して、**変数間の相関**や**影響**を**推定する**ことができます。

- VARモデルは、**複数の変数**に対して、時系列データを表す式を導出することで、それらの変数の間の関係を表すことができます。

---

- 具体的には、VARモデルは、以下の式で表されます。

 - $$ Y_t = c + A_1Y_{t-1} + A_2Y_{t-2} + \dots + A_pY_{t-p} + \varepsilon_t $$


 - $Y_t$は**時刻$t$における変数の値**
 - **$A_1, A_2, \dots, A_p$は係数を表すパラメータ**
 - **$c$は定数**
 - **$\varepsilon_t$は誤差項**
を表します。

- VARモデルでは、**各変数が過去の自身の値**や**他の変数の値**によって**影響を受ける**ことが考えられます。

- VARモデルは、**複数の変数を同時に扱うことができる**ため、**変数間の相関**や**影響を推定する**ことができます。

- また、VARモデルは、**推定された係数**を用いて、**将来の値を予測すること**もできます。

---
- **ベクトル自己回帰（VAR）**とは、**複数の時系列変数間**の**線形相互依存関係**を記述するために用いられる統計モデルである。

- **単一の時系列**のみを考慮する**一変量自己回帰モデル（AR）**を**多変量に拡張したもの**である。

- VARモデルでは、各変数は**自身のラグ値**と**モデル内の他の変数のラグ値**の**一次関数**となる。

- つまり、**各変数の現在値**は、**その過去の値**と、**モデル内の他の変数の過去の値に依存**する。

---
- 以下は、$X$と$Y$の2つの変数を使ったVARモデルの例である。

- $X(t) = a1 * X(t-1) + a2 * Y(t-1) + error(t) Y(t) = b1 * X(t-1) + b2 * Y(t-1) + error(t)$ この例では、$X4 と $Y$ の 2 つの変数からなる $VAR$ モデルの例である。

- この例では、$X$の**現在値**$（X（t））$は、$X$の**前回値**$（X（t-1））$と$Y$の**前回値**$（Y（t-1））$の関数であり、$Y$の**現在値**$（Y（t））$は、$X$の**前回値**$（X（t-1））$と$Y$の**前回値**$（Y（t-1））$の関数である。

- **誤差**$（error(t)）$は、**モデルに含まれない他の要因の影響**を表す。

- $VAR$モデルは、複数の時系列間の**相互依存関係を分析**し、変数の**将来の値について予測**を行うために使用することができます。

- 経済や金融などの分野では、**GDP**や**インフレ**、**雇用**などの**経済指標間の関係**を調べるためによく利用されている。

- 例えば、ある企業の株価$（X）$と原油価格$（Y）$の関係を調べるのに、$VAR$モデルが使われるかもしれません。

- このモデルは、**原油価格の変動**が**株価の変動**に**どの程度影響を与えるか**、あるいは逆にどの程度影響を与えるかを明らかにし、**原油価格の予想変動**に基づいて**将来の株価を予測する**ために使用することができる。

# **状態空間モデル**

---
- 状態空間モデルとは、時系列データを扱うための数理モデルの一種です。

- 状態空間モデルでは、時刻 $t$ の状態を表す状態変数 $x_t$、時刻 $t$ の観測値を表す観測変数 $y_t$、および時刻 $t$ の状態から時刻 $t+14$ の状態への移行を表す移行関数 $f_t4$ と、時刻 $t4$ の状態から時刻 $t$ の観測値への変換を表す観測関数 $h_t$ を用いて、次のような方程式で表されます。

- $x_t = f_t(x_{t-1}, u_t)$

- $y_t = h_t(x_t, v_t)$

- ここで、$u_t$ は時刻 $t$ の状態に対する外部入力を表し、$v_t$ は時刻 $t$ の観測値に対する雑音を表します。

- 状態空間モデルを使用することで、時系列データを効率的に解析することができます。

- また、状態空間モデルを用いて、将来のデータを予測することもできます。

---

- 状態空間モデルとは、**系の振る舞いを表す数学的モデルの一種**です。

- 状態空間モデルは、**状態**、**状態遷移**、**観測**を表す式で構成されます。

- **状態**は、系が持つ情報を表します。
- 例えば、自動車の**状態**は、**スピード、位置、角度など**で表すことができます。
- **状態**は、通常は多次元で表されます。

- **状態遷移**は、**系の状態が時間の経過とともにどのように変化するか**を表します。
- 例えば、自動車の状態遷移は、**スピードによって位置がどのように変化するか**、**角度によってスピードがどのように変化するか**などで表すことができます。

- **観測**は、**系から得られる情報**を表します。
- 例えば、自動車から得られる**観測**は、**車体の位置やスピードなど**です。

- **状態空間モデル**は、**系の状態**や**状態遷移**を**数学的に表す**ことで、**系の振る舞い**を**予測すること**ができます。

- 状態空間モデルは、さまざまな分野で使われており、**自動車の自己走行**や、**工場の生産ライン**などで活用されています。

# **Graph neural network(GNN,グラフ ニューラル ネットワーク)**

---
wiki
- グラフニューラルネットワークについて教えて

---
- グラフ ニューラル ネットワークは、**グラフとして表現できるデータ**を処理するための人工ニューラル ネットワークのクラスです。 

- **「幾何学的深層学習」**のより一般的な主題では、特定の既存のニューラル ネットワーク アーキテクチャは、適切に定義されたグラフで動作する GNN として解釈できます。

---
ChatGPT
- グラフニューラルネットワークについて教えて

---

- グラフニューラルネットワーク (Graph Neural Network, GNN) は、**データを表すグラフ構造上で処理を行うニューラルネットワークのモデル**です。
- **GNN**は、**ノード**や**エッジ**に**属性**や**特徴量**を**割り当て**、それらをmもとに**グラフ全体を表現すること**ができます。

- GNNは、**グラフ構造を持つデータ**を扱うタスクに特に有効です。

 -たとえば、**社会ネットワーク**や**自然言語処理タスク**では、**人**や**単語**などの**データ**を**ノード**として、それらを結ぶ**リンク**や**関係**を**エッジ**として**表現する**ことができます。
 
- **GNN**はこのような**グラフ構造**を利用して、**グラフ全体を表す表現を学習**することができます。

- GNNは、様々なタスクに応用されています。
 - たとえば、**グラフ上でのクラス分類タスク**や**回帰タスク**、**セマンティックセグメンテーションタスク**などが挙げられます。

- また、GNNは**自然言語処理タスク**でも用いられることがあります。

- GNNには、**畳み込みニューラルネットワーク (Convolutional Neural Network, CNN) を拡張したモデル**や、**リカレントニューラルネットワーク (Recurrent Neural Network, RNN)** を拡張したモデルなどがあります。

- これらのモデルは、**グラフ上でのデータ処理を行う**ために、**特別な畳み込み**や**再帰演算を導入**しています。

- GNNは、近年注目されており、様々なタスクで有効な結果が得られています。その一方で、**GNNは計算が複雑である**ことや、**データの欠損**や**ノイズの影響***を**受けやすい**という**欠点**もあります。

---
- グラフニューラルネットワークについてわかりやすく

---

- グラフニューラルネットワーク（Graph Neural Network、GNN）は、**グラフ構造を持つデータ**を扱うためのニューラルネットワークです。

- グラフは、データを**ノード（点）**と**エッジ（辺）**で表すことができます。

- GNNは、**グラフ上**の**ノード同士**をつなぐ**エッジを考慮**して、**各ノードの特徴量を抽出**します。

- GNNは、様々なタスクで活用されています。

- 例えば、**自然言語処理タスク**では、**文章**を**単語**の**グラフ構造**として表現し、GNNを使って**文章の意味を理解する**ことができます。

 - また、**化学データ**や**社会ネットワークデータ**などを扱う場合にも、GNNを使ってそのデータの構造を理解することができます。

- GNNのモデル構造は、普通のニューラルネットワークと大きく異なります。

- 通常のニューラルネットワークでは、**入力データ**を**一つの行列**に**整形**して入力しますが、GNNでは、**各ノード**の情報を**行列として入力**し、その**行列同士**をつなぐ**エッジの情報**も**入力**します。

- その後、**GNN**では、**各ノードの情報**を**更新**するために、その**ノード**とつながっている**エッジの情報**を**考慮した計算**を行います。

- GNNは、**複雑なグラフ構造を扱うタスクでは有効**ですが、**その計算は複雑**であるため、**計算量が大きくなる**ことがあります。

- また、GNNでは、**グラフ構造**を**正確に表現することが重要**であるため、**グラフの構造**を**正しく理解**することが求められます。

---
- グラフニューラルネットワークについて例を用いて説明して

---

- グラフニューラルネットワーク (GNN) は、グラフ構造を扱うためのニューラルネットワークです。

- GNN は、ノードとエッジを含むグラフ構造を入力とし、各ノードの情報を出力します。

- 例えば、ある企業内における**社員のグラフ**を考えます。

 - このグラフでは、**各社員がノード**、**社員間の上司と部下の関係がエッジ**で表されています。
 
 - GNN を用いることで、**各社員のタイトルや役職などの情報**を出力することができます。

 - GNN の計算は、**各ノードから出るエッジ**を辿って、**隣接するノードの情報**を取得し、**各ノードの情報を更新**することで行われます。

- このように、GNN は**グラフ構造上でのデータ伝播を表現**することができます。

- GNN の一般的なアプリケーションとしては、**様々なグラフ構造**を持つデータセットを**分類**や**予測するタスク**、**グラフ上での推論**や**探索タスク**、グラフ上での**ノード分類**や**エッジ生成タスク**などがあります。

---
- グラフニューラルネットワークについて他の例を用いて説明して

- グラフニューラルネットワーク (Graph Neural Network, GNN) は、グラフ構造を持つデータを扱うためのニューラルネットワークモデルです。

- 典型的な使用例としては、以下のようなものがあります。

- **化学反応における分子の構造**や**反応性を予測する**ためのモデル構築

- **ソーシャルネットワークや航空ルート**のような、**ノードとエッジで構成されるネットワークを分析する**ためのモデル構築

- 自然言語処理タスクで、**文章や文書を構成する単語間の関係**を表すグラフを扱うためのモデル構築

- **医学画像解析で、解剖学的構造を表すグラフ**を扱うためのモデル構築

- GNN は、通常のニューラルネットワークと同様に、**入力データから出力を予測するモデル**ですが、**入力データ**が**グラフ構造を持つこと**が特徴です。

- GNN は、**グラフ上のノードやエッジに対応する特徴量**を学習し、それらの**特徴量**を元に**グラフ全体の特徴を抽出**することができます。

- GNN の実装には、複数の種類のモデルがあります。

- その一つが、**畳み込みニューラルネットワーク (Convolutional Neural Network, CNN)** を拡張したものです。

- このモデルでは、グラフ上のノードやエッジに対応する**特徴量を畳み込み層**を通じて学習します。

- 他にも、**リカレントニューラルネットワーク (Recurrent Neural Network, RNN)** を拡張したものや、**マルチヘッド注意機構 (Multi-Head Attention Mechanism)** を用いたものなどがあります。

# **ミニマックス戦略**

---
ChatGPT
- ミニマックス戦略について例を出してわかりやすく教えて
---
- ミニマックス戦略とは、ゲーム理論において、**プレイヤーが自分ができる中で最も悪い結果を最小化する戦略をとること**を指します。

- この戦略は、**相手がどのような戦略**をとっても、**自分ができる限り最も悪い結果**を**最小化**することで、**最も良い結果を得ることができる**と考えられています。

- 例えば、二人でカードゲームをするとします。

---
- このゲームでは、**二人とも 1 枚のカードを引き、カードの大小で勝負をする**ものとします。

- このとき、**プレイヤー A** は、**自分ができる限り最も悪い結果を最小化する**ために、**相手が引くカードの大小を考慮**して、**自分が引くカードを選ぶ**ことにします。

- つまり、**相手が引くカードが大きい場合は、小さいカードを引く**ことで、**自分ができる限り最も悪い結果を最小化**し、**最も良い結果を得ることができます。**

- これが、**ミニマックス戦略**の例です。ゲーム理論においては、このような戦略をとることで、**最も良い結果を得ることができる**と考えられています。

---
ChatGPT
- Tell me about the minimax strategy in a simple way with examples.
- （訳）ミニマックス戦略について例を出してわかりやすく教えて
---
- ミニマックス戦略とは、チェス、三目並べ、囲碁などの**二人用ゲーム**で用いられる**意思決定アルゴリズムの一つ**である。

- プレイヤーにとって**最大の損失を最小限に抑えるという考え**に基づいており、**"ミニマックス原理 "**とも呼ばれる。

- その仕組みはこうだ。

---
- **2人用のゲーム**では、各プレイヤーは**各ターンで可能な限り最善の手を打つ**ことで、**勝つチャンスを最大化**しようとする。

- 同時に、各プレイヤーは、**相手の可能な動きを予測**し、**それに対抗する**ことによって、**相手の勝利のチャンスを最小**にしようとする。

- そのために、ミニマックスアルゴリズムは、**すべての可能な手**と**その結果**を、**手を打つプレーヤー**と**相手**の**両方について検討**する。

- そして、**プレーヤーにとって損失が最小**になり、**相手にとっても損失が最大**になるような手を選択する。

- 三目並べのような簡単なゲームでミニマックス戦略がどのように機能するか、例を挙げてみましょう。

- プレイヤー1（X）が最初の手を打ち、ボードの左上隅に自分のXを置きます。

- プレイヤー2（O）は、自分のOをどこに置くかを決めなければなりません。ミニマックス戦略を使って、可能なすべての手とその結果を検討します。

- 例えば、プレイヤー2が○をボードの中央に置いた場合、プレイヤー1は○を右上と左下に置けば勝てる可能性があります。

- **プレイヤー2はその損失を最小限**にするため、右上に○を置き、**プレイヤー1の勝利の可能性を阻み**、**自分の勝利の可能性を設定する**ことができるのです。

- ミニマックスアルゴリズムは、**すべての可能な手**と**その結果を考慮**することで、**プレイヤーが勝つ可能性を最大**にし、**負ける可能性を最小**にする**戦略的決定を行うこと**を支援します。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import hydra

from omegaconf import DictConfig
from omegaconf.omegaconf import OmegaConf

import numpy as np
from sklearn.datasets import load_digits

import matplotlib.pyplot as plt



In [ ]:
time_series = np.random.rand(1000, 300)
time_series[500:560, 100:200] += 0.3
time_series = torch.from_numpy(time_series)

In [ ]:
p = torch.from_numpy(np.abs(np.indices((100,100))[0] - np.indices((100,100))[1]))
sigma = torch.ones(100).view(100, 1) * 2

In [ ]:
P = torch.ones(10,10) * torch.arange(10).view(10,1)

In [ ]:
S = torch.ones(10,10) * torch.arange(10).view(1,10)

In [ ]:
lambda row: F.kl_div(P[row,:], S[row,:]) + F.kl_div(S[row,:], P[row,:])

/Users/spencerbraun/.pyenv/versions/3.8.11/envs/dl/lib/python3.8/site-packages/torch/nn/functional.py:2747: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


tensor(3.4057)

In [ ]:
P

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2., 2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3., 3., 3., 3., 3., 3.],
        [4., 4., 4., 4., 4., 4., 4., 4., 4., 4.],
        [5., 5., 5., 5., 5., 5., 5., 5., 5., 5.],
        [6., 6., 6., 6., 6., 6., 6., 6., 6., 6.],
        [7., 7., 7., 7., 7., 7., 7., 7., 7., 7.],
        [8., 8., 8., 8., 8., 8., 8., 8., 8., 8.],
        [9., 9., 9., 9., 9., 9., 9., 9., 9., 9.]])

In [ ]:
torch.arange(0, 10, 0.1)

tensor([0.0000, 0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000, 0.7000, 0.8000,
        0.9000, 1.0000, 1.1000, 1.2000, 1.3000, 1.4000, 1.5000, 1.6000, 1.7000,
        1.8000, 1.9000, 2.0000, 2.1000, 2.2000, 2.3000, 2.4000, 2.5000, 2.6000,
        2.7000, 2.8000, 2.9000, 3.0000, 3.1000, 3.2000, 3.3000, 3.4000, 3.5000,
        3.6000, 3.7000, 3.8000, 3.9000, 4.0000, 4.1000, 4.2000, 4.3000, 4.4000,
        4.5000, 4.6000, 4.7000, 4.8000, 4.9000, 5.0000, 5.1000, 5.2000, 5.3000,
        5.4000, 5.5000, 5.6000, 5.7000, 5.8000, 5.9000, 6.0000, 6.1000, 6.2000,
        6.3000, 6.4000, 6.5000, 6.6000, 6.7000, 6.8000, 6.9000, 7.0000, 7.1000,
        7.2000, 7.3000, 7.4000, 7.5000, 7.6000, 7.7000, 7.8000, 7.9000, 8.0000,
        8.1000, 8.2000, 8.3000, 8.4000, 8.5000, 8.6000, 8.7000, 8.8000, 8.9000,
        9.0000, 9.1000, 9.2000, 9.3000, 9.4000, 9.5000, 9.6000, 9.7000, 9.8000,
        9.9000])

In [ ]:
torch.exp(p.pow(2) / (sigma))

NameError: name 'sigma' is not defined

In [ ]:
gaussian = torch.normal(p.float(), sigma)
gaussian /= gaussian.sum(dim=-1).view(-1, 1)

NameError: name 'p' is not defined

In [ ]:
gaussian[0,:].sum()

NameError: name 'gaussian' is not defined

In [ ]:
torch.ones(5,10)/ torch.ones(5,10).sum(dim=-1).view(-1,1)

tensor([[0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000]])

In [ ]:
torch.ones(5,10).sum(dim=-1)

tensor([10., 10., 10., 10., 10.])

In [ ]:
torch.normal(torch.arange(0,100).float())

tensor([3.6336e-02, 7.9632e-01, 1.7809e+00, 3.8046e+00, 2.3193e+00, 4.2680e+00,
        8.3718e+00, 7.3727e+00, 6.7060e+00, 1.1051e+01, 1.0537e+01, 1.0064e+01,
        1.1236e+01, 1.1610e+01, 1.4704e+01, 1.5483e+01, 1.6960e+01, 1.7324e+01,
        1.8780e+01, 1.9655e+01, 1.9641e+01, 2.2539e+01, 2.2296e+01, 2.1461e+01,
        2.3893e+01, 2.5538e+01, 2.7476e+01, 2.7799e+01, 2.8607e+01, 3.0070e+01,
        2.9921e+01, 3.0479e+01, 3.2850e+01, 3.2870e+01, 3.5183e+01, 3.4523e+01,
        3.5027e+01, 3.6092e+01, 3.7668e+01, 3.8510e+01, 4.0233e+01, 4.1136e+01,
        4.1330e+01, 4.4310e+01, 4.5134e+01, 4.6767e+01, 4.7380e+01, 4.6682e+01,
        4.8102e+01, 4.8557e+01, 5.0157e+01, 5.1855e+01, 5.2537e+01, 5.3827e+01,
        5.4526e+01, 5.5917e+01, 5.6168e+01, 5.6827e+01, 5.9424e+01, 5.6755e+01,
        5.9634e+01, 6.0564e+01, 5.9774e+01, 6.2367e+01, 6.1566e+01, 6.5457e+01,
        6.7245e+01, 6.8038e+01, 6.8918e+01, 7.0426e+01, 7.0787e+01, 7.1730e+01,
        7.2374e+01, 7.3203e+01, 7.5236e+

In [ ]:
class AnomalyAttention(nn.Module):
    def __init__(self, seq_dim, in_channels, out_channels):
        super(AnomalyAttention, self).__init__()
        self.W = nn.Linear(in_channels, out_channels, bias=False)
        self.Q = self.K = self.V = self.sigma = torch.zeros((seq_dim, out_channels))
        self.d_model = out_channels
        self.n  = seq_dim
        self.P = torch.zeros((seq_dim, seq_dim))
        self.S = torch.zeros((seq_dim, seq_dim))

    def forward(self, x):

        self.initialize(x) # does this make sense?
        self.P = self.prior_association()
        self.S = self.series_association()
        Z = self.reconstruction()

        return Z

    def initialize(self, x):
        # self.d_model = x.shape[-1]
        self.Q = self.K = self.V = self.sigma = self.W(x)
        

    def prior_association(self):
        p = torch.from_numpy(
            np.abs(
                np.indices((self.n,self.n))[0] - 
                np.indices((self.n,self.n))[1]
                )
            )
        gaussian = torch.normal(p.float(), self.sigma[:,0].abs())
        gaussian /= gaussian.sum(dim=-1).view(-1, 1)

        return gaussian

    def series_association(self):
        return F.softmax((self.Q @ self.K.T) / math.sqrt(self.d_model), dim=0)

    def reconstruction(self):
        return self.S @ self.V

    def association_discrepancy(self):
        return F.kl_div(self.P, self.S) + F.kl_div(self.S, self.P) #not going to be correct dimensions



In [ ]:
class AnomalyTransformerBlock(nn.Module):
    def __init__(self, seq_dim, feat_dim):
        super().__init__()
        self.seq_dim, self.feat_dim = seq_dim, feat_dim
       
        self.attention = AnomalyAttention(self.seq_dim, self.feat_dim, self.feat_dim)
        self.ln1 = nn.LayerNorm(self.feat_dim)
        self.ff = nn.Sequential(
            nn.Linear(self.feat_dim, self.feat_dim),
            nn.ReLU()
        )
        self.ln2 = nn.LayerNorm(self.feat_dim)
        self.association_discrepancy = None

    def forward(self, x):
        x_identity = x 
        x = self.attention(x)
        z = self.ln1(x + x_identity)
        
        z_identity = z
        z = self.ff(z)
        z = self.ln2(z + z_identity)

        self.association_discrepancy = self.attention.association_discrepancy().detach()
        
        return z

In [ ]:
class AnomalyTransformer(nn.Module):
    def __init__(self, seqs, in_channels, layers, lambda_):
        super().__init__()
        self.blocks = nn.ModuleList([
            AnomalyTransformerBlock(seqs, in_channels) for _ in range(layers)
        ])
        self.output = None
        self.lambda_ = lambda_
        self.assoc_discrepancy = torch.zeros((seqs, len(self.blocks)))
    
    def forward(self, x):
        for idx, block in enumerate(self.blocks):
            x = block(x)
            self.assoc_discrepancy[:, idx] = block.association_discrepancy
        
        self.assoc_discrepancy = self.assoc_discrepancy.sum(dim=1) #N x 1
        self.output = x
        return x

    def loss(self, x):
        l2_norm = torch.linalg.matrix_norm(self.output - x, ord=2)
        return l2_norm + (self.lambda_ * self.assoc_discrepancy.mean())

    def anomaly_score(self, x):
        score = F.softmax(-self.assoc_discrepancy, dim=0)

In [ ]:
model = AnomalyTransformer(seqs=1000, in_channels=300, layers=3, lambda_=0.1)

In [ ]:
model(time_series.float())

/Users/spencerbraun/.pyenv/versions/3.8.11/envs/dl/lib/python3.8/site-packages/torch/nn/functional.py:2747: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


tensor([[-2.2100,  0.2870, -0.2157,  ..., -1.2898, -1.4303, -0.3504],
        [-1.1387, -0.4830, -0.9881,  ..., -1.1120, -2.4669, -0.5230],
        [-0.3306, -0.7025, -1.5827,  ..., -0.3628, -1.8427,  0.7217],
        ...,
        [-0.5868, -0.5519, -1.9108,  ..., -0.4716, -2.8175, -1.0170],
        [-1.6752, -0.7690, -2.3892,  ..., -1.6920, -2.6238,  0.9201],
        [-1.0995, -0.0956, -0.5864,  ..., -2.5304, -2.2143, -0.5381]],
       grad_fn=<NativeLayerNormBackward0>)

In [ ]:
model.loss(time_series)

tensor(508.2613, dtype=torch.float64, grad_fn=<AddBackward0>)